# Résoudre les élisions fautives

## L’élision en français

**Élision :** mécanisme d’effacement de la voyelle finale d’un mot devant la voyelle du mot suivant, matérialisée par une apostrophe (`’`).

Phénomène courant, parfois obligatoire (1), facultatif (2) ou totalement fautif (3) :
```txt
(1) *Le arbre est grand.
(2) Arthur : Et dis donc machin, tu fais péter du raisin,
    t'es gentil !
(3) Le tavernier : Ah heu Sire, j'vais pas vous raconter
    d'conneries.
```

**Rappel :** la retranscription des scripts de la série ne suit aucune directive. Volonté des transcripteurs de reproduire à l’écrit ce qu’ils entendent à l’oral.

**Objectif :** rétablir les formes improprement élidées (4).
```txt
(4) Le tavernier : Ah heu Sire, je vais pas vous raconter
    de conneries.
```

## Méthodologie

Détecter les élisions fautives avant de les résoudre !

Recours à une expression rationnelle pour détecter les motifs non autorisés.

**Motif autorisé :** un mot suivi d’une apostrophe qui soit lui-même suivi d’un mot débutant par une voyelle.  
**Motif non autorisé :** un mot suivi d’une apostrophe qui **ne** soit lui-même **pas** suivi d’un mot débutant par une voyelle.

Exemple de détection des élisions autorisées :

In [ ]:
import re

# Pattern to detect elisions
allowed = re.compile(r'\w+[\'’][aeiouyhéèêìîôöùûüAEIOUYHÉÈÊÌÎÔÖÛÜÙ]\w+')

# Reading a file
with open(f'kaamelott/sample/S01E01-heat.txt') as file:
    text = file.read()

results = allowed.findall(text)
print(results)

Pour détecter les élisions fautives, modifier légèrement la *regex* :

In [ ]:
wrong = re.compile(r'\w+[\'’][^aeiouyhéèêìîôöùûüAEIOUYHÉÈÊÌÎÔÖÛÜÙ]\w+')
results = wrong.findall(text)
print(results)

Signe `^` pour exclure les caractères autorisés après une élision.

Pour résoudre les élisions : transformer le signe `'` en `e` et rajouter une espace.

Utiliser les parenthèses pour capturer dans la *regex* des parties à réutiliser :

In [ ]:
pattern = re.compile(r'(\w+)[\'’]([^aeiouyhéèêìîôöùûüAEIOUYHÉÈÊÌÎÔÖÛÜÙ]\w+)')
results = pattern.findall(text)
print(results)

Pour déplier les tuples de résultats :

In [ ]:
for (elided_shape, following_word) in results:
    print(f'{elided_shape}e {following_word}', end=', ')

Quelques cas particuliers :
```txt
(5) de vant
(6) deve nir
```

Ces cas concernent les élisions internes à un mot, qui appellent un traitement manuel.

Autre considération à prendre en compte dans la méthodologie, le dédoublonnage des résultats. On comptabilise trois occurrences des formes : `s'trouve`.

Structure de données `set` pour les fusionner

In [ ]:
elisions = set()
[elisions.add(result) for result in results]
for (elided_shape, following_word) in results:
    print(f'{elided_shape}e {following_word}', end=', ')

En prévision, certaines élisions ambiguës :
```txt
(7) *d’mander
```

Cette forme pourrait aboutir à deux solutions :
- `demander`
- `de mander`

Les contextes gauche et droite seraient utiles pour décider. Adapter la *regex* en ce sens :

In [ ]:
pattern = re.compile(r'(\w+)\W+?(\w+)[’\']([^aeiouyhéèêìîôöùûüAEIOUYHÉÈÊÌÎÔÖÛÜÙ]\w+)\W+?(\w+)?')
results = pattern.findall(text)
print(len(results))
for (left_context, elided_shape, following_shape, right_context) in results:
    print(f'{left_context}\t[{elided_shape} {following_shape}]\t{right_context}')

**Attention !** Conflit dans la méthodologie :
- structure `set` fusionne les résultats de la recherche sur les élisions fautives
- un résultat de cette recherche pourrait correspondre à deux réalités (*demander* vs *de mander*)

Face à une élision fautive, deux hypothèses :
- élision par suppression de la voyelle finale
- élision interne au mot

Une solution consisterait à :
1. construire un fichier des élisions fautives à résoudre
2. confronter la seconde hypothèse avec un lexique
3. traiter différemment les formes qui vérifient l’une ou l’autre des hypothèses

## Algorithme pas à pas

**Étape 1 :** importer le module `re`

In [1]:
import re

**Étape 2 :** détecter les élisions fautives dans un texte

In [2]:
with open(f'kaamelott/sample/S01E01-heat.txt') as file:
    text = file.read()

pattern = re.compile(r'(\w+)\W+?(\w+)[’\']([^aeiouyhéèêìîôöùûüAEIOUYHÉÈÊÌÎÔÖÛÜÙ]\w+)\W+?(\w+)?')
results = pattern.findall(text)

**Étape 3 :** confronter chaque résultat avec un lexique de référence

**Étape 4 :** écrire un fichier recensant les contextes gauche et droite en prévision d’un arbitrage manuel de résolution des élisions fautives

In [5]:
with open('kaamelott/elisions_context.txt', 'w') as file:
    file.write('S01E01-heat\n')
    for (left_context, elided_shape, following_shape, right_context) in results:
        file.write(f'{left_context}\t[{elided_shape}’{following_shape}]\t{right_context}\n')

In [ ]:
elisions = list()
lexique = ['demandé']
cue = 'Me suis d’mandé comment il s’faisait qu’il avait la tête à l’envers.'
pattern = re.compile(r'(\w+)[\'’]([^aeiouyhéèêìîôöùûüAEIOUYHÉÈÊÌÎÔÖÛÜÙ]\w+)')
results = pattern.findall(cue)
for (elided_shape, following_shape) in results:
    word = f'{elided_shape}e{following_shape}'
    if word not in lexique:
        elisions.append((elided_shape, following_shape))
print(elisions)